In [2]:
import numpy as np
import sklearn.linear_model as skl



class RegressionMethods:
    """
    Class implementation of the OLS, Ridge and Lasso regression methods.
    Initiate with a specification of which method to use, and hyperparameter alpha for Ridge and Lasso.
    The OLS and Ridge methods find the analytical beta-parameters, the Lasso method is a call to
    sklearns Lasso method.
    """

    def __init__(self, method = 'ols', alpha = 0):
        self.method = method
        self.alpha = alpha


    def ols(self):
        """
        Finds the beta-parameters by OLS regression.
        In case of singular matrix, uses the pseudo inverse in
        calculation of beta.
        """
        XT = self.X.T
        self.beta = np.linalg.pinv(XT.dot(self.X)).dot(XT).dot(self.z)


    def ridge(self):
        """
        Finds the beta-parameters by Ridge regression.
        In case of singular matrix, uses the pseudo inverse in
        calculation of beta.
        """
        XT = self.X.T
        p = np.shape(self.X)[1]
        L = np.identity(p)*self.alpha
        self.beta = np.linalg.pinv(XT.dot(self.X) + L).dot(XT).dot(self.z)


    def lasso(self):
        clf = skl.Lasso(alpha = self.alpha, fit_intercept=False, normalize=False, max_iter=10000, tol=0.006).fit(self.X, self.z)
        self.beta = clf.coef_


    def fit(self, X, z):
        """
        Fits the specified model to the data. Makes a call to the
        relevant regression method.
        Inputs:
        -Design matrix X, dimension (n, p)
        -Target values z, dimension (n, 1)
        """
        self.X = X
        self.z = z
        if self.method == 'ols':
            self.ols()
        elif self.method == 'ridge':
            self.ridge()
        elif self.method == 'lasso':
            self.lasso()


    def predict(self, X):
        """
        Does a prediction on a set of data with the
        parameters beta found by the fit-method.
        Input:
        -Data to be predicted on, in a matrix (n, p)
        """
        self.z_tilde = X @ self.beta
        return self.z_tilde


    def set_alpha(self, alpha):
        """
        Change the alpha parameter after initialiation.
        """
        self.alpha = alpha

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import time


class Resampling:
    """
    Class for performing a train/test split and bootstrap resampling.
    Is initialized with the whole data set, splitting in training and testing
    data is performed by the methods.
    """
    def __init__(self, X, z):
        self.X = X.astype('float64')
        self.z = z.astype('float64')


    def train_test(self, model, test_size = 0.2):
        """
        Performs a simple train/test split and trains a model on the train data
        and returns the errors of the predictions on the test data.
        Inputs:
        -model, instanciated model
        -test_size, how much of the data to be used in testing
        """


        #split the data in training and test
        X_train, X_test, z_train, z_test = train_test_split(self.X, self.z, test_size = test_size)

        #fit the model on the train data
        model.fit(X_train, z_train)

        #predict on the test data
        z_pred = model.predict(X_test)

        #calculate errors
        error = np.mean((z_test - z_pred)**2)
        bias = np.mean((z_test - np.mean(z_pred))**2)
        variance = np.var(z_pred)
        r2 = r2_score(z_test, z_pred)

        return error, bias, variance, r2


    def bootstrap(self, model, n_bootstraps = 100, test_size = 0.2, get_beta_var = False):
        """
        Performs bootstrap resampling and returns the error scores of both training and
        test data. Can also return the beta-parameters with its variance if specified.
        Inputs:
        -model, instanciated model
        -n_bootstraps, how many bootstrap resamplings to perform
        -test_size, how much of the data to be used in testing
        -get_beta_var, whether to return only beta values and their variance
        """

        #split data in training and testing
        X_train, X_test, z_train, z_test = train_test_split(self.X, self.z, test_size = test_size)
        sampleSize = X_train.shape[0]
        n_betas = np.shape(self.X)[1]

        #setup arrays for storing prediction values
        z_pred = np.empty((z_test.shape[0], n_bootstraps))
        z_train_pred = np.empty((z_train.shape[0], n_bootstraps))
        z_train_boot = np.empty((z_train.shape[0], n_bootstraps))
        betas = np.empty((n_betas, n_bootstraps))
        r2 = np.empty(n_bootstraps)


        #perform the resamplings
        for i in range(n_bootstraps):

            #pick random values in the training data and fit the model to it
            indices = np.random.randint(0, sampleSize, sampleSize)
            X_, z_ = X_train[indices], z_train[indices]
            model.fit(X_, z_)

            #save z-values of the training set
            z_train_boot[:,i] = z_

            #predict on the same test data each time
            z_pred[:,i] = model.predict(X_test)
            z_train_pred[:,i] = model.predict(X_)
            betas[:,i] = model.beta
            r2[i] = r2_score(z_pred[:,i], z_test)


        z_test = z_test.reshape((len(z_test), 1))

        #calculate mean error scores
        error = np.mean( np.mean((z_pred - z_test)**2, axis=1, keepdims=True))
        error_train = np.mean( np.mean((z_train_pred - z_train_boot)**2, axis=1, keepdims=True))
        bias = np.mean( (z_test - np.mean(z_pred, axis=1, keepdims=True))**2 )
        variance = np.mean( np.var(z_pred, axis=1, keepdims=True) )


        beta_variance = np.var(betas, axis=1)
        betas = np.mean(betas, axis=1)


        if get_beta_var:
            return betas, beta_variance
        else:
            return error, bias, variance, error_train, np.mean(r2)

In [5]:
import numpy as np
import pandas as pd
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from imageio import imread
import seaborn as sns


def load_terrain(filename):
    """
    Function to load terrain data and convert it to a 2D array
    Returns a rescaled version of the array and its dimensions.
    """
    slice = 2
    terrain = imread('data/' + filename)
    dims = np.shape(terrain)
    print(dims)
    if dims[0] != dims[1]:
        terrain = terrain[0:dims[1], :]
        dims = terrain.shape
    terrain = terrain[0:dims[0]//2, 0:dims[1]//2]
    terrain = terrain[0:-1:slice, 0:-1:slice]
    dims = np.shape(terrain)
    print(filename, 'loaded.', dims[0],'x',dims[1])
    return terrain*0.001, dims[0]


def show_terrain(terrain_data):
    """
    Simple function for showing the raw terrain data
    """
    terrain1 = imread(terrain_data)
    plt.figure()
    plt.title('Terrain')
    plt.imshow(terrain1, cmap='gray')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()


def generate_mesh(n, random_pts = 0):
    """
    Generated a mesh of n x and y values.
    """
    if random_pts == 0:
        x = np.linspace(0, 1, n)
        y = np.linspace(0, 1, n)
    if random_pts == 1:
        x = np.random.rand(n)
        y = np.random.rand(n)
    return np.meshgrid(x, y)


def frankie_function(x, y, n, sigma = 0, mu = 0):
    """
    Calculates the values of the Franke function.
    Returns these values with an element of noise if specified.
    """
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4 + np.random.normal(mu, sigma, n)


def create_design_matrix(x, y, deg):
    """
    Creates a design matrix with columns:
    [1  x  y  x^2  y^2  xy  x^3  y^3  x^2y ...]
    """
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    p = int((deg + 1)*(deg + 2)/2)
    X = np.ones((N,p))

    for i in range(1, deg + 1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = x**(i-k) * y**k

    return X



def plot_model(x, y, z, model, deg):
    """
    Function for plotting a height map of the predicted output values of a model,
    and the raw terrain data.
    Inputs:
    -model, instanciated model
    -deg, degree of the model
    """

    #perform regression on the data and predict
    X = create_design_matrix(x, y, deg)
    z_flat = np.ravel(z)
    model.fit(X, z_flat)
    z_pred = model.predict(X).reshape(len(x), len(y))

    #plotting
    plt.rcParams.update({'font.size': 14})
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1, projection='3d', gridspec_kw = {'wspace':0, 'hspace':0})
    ax2 = fig.add_subplot(1, 2, 2, projection='3d', gridspec_kw = {'wspace':0, 'hspace':0})
    surf1 = ax1.plot_surface(x, y, z_pred, cmap=cm.terrain, alpha=0.9)
    surf2 = ax2.plot_surface(x, y, z, cmap=cm.terrain, alpha=0.9)
    ax1.set_zlim(0.75, 1.75)
    ax2.set_zlim(0.75, 1.75)
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('Height [km]')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_zlabel('Height [km]')
    ax1.title.set_text('Modelled terrain')
    ax2.title.set_text('Actual terrain')
    plt.show()

In [6]:
import pandas as pd
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from matplotlib.patches import Rectangle
import time
plt.style.use('ggplot')





def model_degree_analysis(x, y, z, model_name, min_deg=1, max_deg=10, n_bootstraps = 100, alpha = 0, ID = '000'):
    """
    Function for analyzing the performance of a model for different model complexities.
    Performs bootstrap resampling for each configuration.
    Plots the MSE of the training error and test error for each configuration,
    and the bias^2 - variance decomposition of the testing error.
    The error scores for each configuration is also saved to a .csv file.
    Inputs:
    -x, y values, dimensions (n, n)
    -z values, dimension (n^2, 1)
    -model_name, name of the model: 'ols', 'ridge', 'lasso'
    -min_deg, max_deg - degrees to analyze
    -n_bootstraps - number of resamples in bootstrap
    -alpha - hyperparameter in 'ridge' and 'lasso'
    -ID - figure IDs
    """


    #setup directories
    dat_filename = 'results/' + 'error_scores_deg_analysis_' + model_name
    fig_filename = 'figures/' + 'deg_analysis_' + model_name
    error_scores = pd.DataFrame(columns=['degree', 'mse', 'bias', 'variance', 'r2', 'mse_train'])

    #initialize regression model and arrays for saving error values
    model = RegressionMethods(model_name, alpha=alpha)
    degrees = np.linspace(min_deg, max_deg, max_deg - min_deg + 1)
    nDegs = len(degrees)
    mse = np.zeros(nDegs)
    bias = np.zeros(nDegs)
    variance = np.zeros(nDegs)
    r2 = np.zeros(nDegs)
    mse_train = np.zeros(nDegs)


    min_mse = 1e100
    min_r2 = 0
    min_deg = 0
    i = 0

    #loop through the specified degrees to be analyzed
    for deg in degrees:
        X = create_design_matrix(x, y, int(deg))
        resample = Resampling(X, z)

        #perform bootstrap resampling and save error values
        mse[i], bias[i], variance[i], mse_train[i], r2[i] = resample.bootstrap(model, n_bootstraps)

        #save to pandas dataframe
        error_scores = error_scores.append({'degree': degrees[i],
                                            'mse': mse[i],
                                            'bias': bias[i],
                                            'variance': variance[i],
                                            'r2': r2[i],
                                            'mse_train': mse_train[i]}, ignore_index=True)

        #check if this configuration gives smallest error
        if mse[i] < min_mse:
            min_mse = mse[i]
            min_r2 = r2[i]
            min_deg = deg


        i += 1
    #end for


    #plot error of test set and training set
    plt.plot(degrees, mse, label='test set')
    plt.plot(degrees, mse_train, label='training set')
    plt.legend()
    plt.xlabel('Model complexity [degree]')
    plt.ylabel('Mean Squared Error')
    plt.savefig(fig_filename + '_test_train_' + ID + '.pdf')
    plt.show()

    #plot bias^2 variance decomposition of the test error
    plt.plot(degrees, mse, label='mse')
    plt.plot(degrees, bias,'--', label='bias')
    plt.plot(degrees, variance, label='variance')
    plt.xlabel('Model complexity [degree]')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.savefig(fig_filename + '_bias_variance_' + ID + '.pdf')
    plt.show()


    #save error scores to file
    error_scores.to_csv(dat_filename + '.csv')
    print('min mse:', min_mse)
    print('r2:', min_r2)
    print('deg:', min_deg)



def ridge_lasso_complexity_analysis(x, y, z, model_name, min_deg=1, max_deg=10, n_lambdas=13, min_lamb=-10, max_lamb=2, ID = '000'):
    """
    Function for analyzing ridge or lasso model performance for
    different values lambda and model complexity.
    Performs bootstrap resampling for each configuration and plots
    a heat map of the error for each configuration. Error scores are
    also saved to a .csv file.
    Inputs:
    -x, y values, dimensions (n, n)
    -z values, dimension (n^2, 1)
    -model_name, name of the model: 'ridge', 'lasso'
    -min_deg, max_deg - degrees to analyze
    -min_lamb, max_lamb, n_lambdas - values of log_10(lambda) to analyze, and how many
    -ID - figure IDs
    """


    #initialize model and arrays for parameters lambda and complexity
    model = RegressionMethods(model_name)
    lambdas = np.linspace(min_lamb, max_lamb, n_lambdas)
    degrees = np.linspace(min_deg, max_deg, max_deg - min_deg + 1)

    #setup directories
    dat_filename = 'results/' + 'error_scores_' + model_name
    fig_filename = 'figures/' + 'min_mse_meatmap_' + model_name
    error_scores = pd.DataFrame(columns=['degree', 'log lambda', 'mse', 'bias', 'variance', 'r2', 'mse_train'])


    min_mse = 1e100
    min_lambda = 0
    min_degree = 0
    min_r2 = 0


    i = 0
    #loop through specified degrees
    for deg in degrees:
        j = 0
        X = create_design_matrix(x, y, int(deg))
        resample = Resampling(X, z)
        #loop through specified lambdas
        for lamb in tqdm(lambdas):
            model.set_alpha(10**lamb)

            #perform resampling
            mse, bias, variance, mse_train, r2 = resample.bootstrap(model, n_bootstraps=10)

            #save error scores in pandas dataframe
            error_scores = error_scores.append({'degree': deg,
                                                'log lambda': lamb,
                                                'mse': mse,
                                                'bias': bias,
                                                'variance': variance,
                                                'r2': r2,
                                                'mse_train': mse_train}, ignore_index=True)

            #check if current configuration gives minimal error
            if mse < min_mse:
                min_mse = mse
                min_lambda = lamb
                min_degree = deg
                min_r2 = r2

            j+=1
        #end for lambdas
        i+=1
    #end for degrees

    print('min mse:', min_mse)
    print('min r2:', min_r2)
    print('degree:', min_degree)
    print('lambda:', min_lambda)


    #save scores to file
    error_scores.to_csv(dat_filename + '.csv')



    #plot heat map of error scores of each configuration
    mse_table = pd.pivot_table(error_scores, values='mse', index=['degree'], columns='log lambda')
    idx_i = np.where(mse_table == min_mse)[0]
    idx_j = np.where(mse_table == min_mse)[1]

    fig = plt.figure()
    ax = sns.heatmap(mse_table, annot=True, fmt='.2g', cbar=True, linewidths=1, linecolor='white',
                            cbar_kws={'label': 'Mean Squared Error'})
    ax.add_patch(Rectangle((idx_j, idx_i), 1, 1, fill=False, edgecolor='red', lw=3))

    ax.set_xlabel(r"$\log_{10}(\lambda)$")
    ax.set_ylabel("Complexity")
    ax.set_ylim(len(degrees), 0)
    plt.show()





def confidence_intervals(x, y, z_flat, model, degree, alpha = 0, noise = 0):
    """
    Function for finding the estimated confidence intervals of a given models beta-parameters,
    and makes a plot of the parameters with confidence intervals corresponing to
    a 95% confidence interval.
    """
    X = create_design_matrix(x, y, degree)
    resample = Resampling(X, z_flat)
    betas, variance = resample.bootstrap(model, get_beta_var=True)

    CI = 1.96*np.sqrt(variance)


    #plotting
    plt.xticks(np.arange(0, len(betas), step=1))
    plt.errorbar(range(len(betas)), betas, CI, fmt="b.", capsize=3, label=r'$\beta_j \pm 1.96 \sigma$')
    plt.legend()
    plt.xlabel(r'index $j$')
    plt.ylabel(r'$\beta_j$')
    plt.grid()
    plt.show()



def confidence_interval_ols(X, z):
    """
    Function for finding the confidence intervals of the OLS beta-parameters.
    Uses the analytical expression of the variance. Makes a plot of the beta-parameters with
    error bars corresponing to 95% confidence interval.
    """

    model = RegressionMethods('ols')
    model.fit(X, z)
    betas = model.beta
    cov = np.var(z)*np.linalg.pinv(X.T.dot(X))
    std_betas = np.sqrt(np.diag(cov))
    CI = 1.96*std_betas


    #plot results
    plt.errorbar(range(len(betas)), betas, CI, fmt="b.", capsize=3, label=r'$\beta_j \pm 1.96 \sigma$')
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.legend()
    plt.xlabel(r'index $j$')
    plt.ylabel(r'$\beta_j$')
    plt.grid()
    plt.show()

In [7]:
#performing the analysis of the regression methods
np.random.seed(100)
n = 20
deg = 5
sigma = 0.2

In [8]:
# frankie data
x, y = generate_mesh(n)
z = frankie_function(x, y, n, sigma)
z_flat = np.ravel(z)

In [11]:
# terrain data
terrain_data, n = load_terrain('norway1.tif')
z_flat = np.ravel(terrain_data)
x, y = generate_mesh(n)


FileNotFoundError: No such file: 'C:\Users\bod\OneDrive - Universitetet i Oslo\DL and ML\Project 1\data\norway1.tif'

In [12]:
class UnitTests():
    """
    Tests the OLS and Ridge implementations in the RegressionMethods class, by
    comparing its results with sklearns equivalent methods.
    A test case is initialized automatically in the __init__ function.
    """

    def __init__(self):
        self.n = 100
        np.random.seed(10)
        degree = 5
        sigma = 0.3
        x, y = generate_mesh(self.n)
        z = frankie_function(x, y, self.n, sigma)
        self.z_flat = np.ravel(z)
        self.X = create_design_matrix(x, y, degree)
        self.tol = 1e-15

    def test_ols(self):

        #perform ols on the data with sklean
        clf = skl.LinearRegression().fit(self.X, self.z_flat)
        z_pred_skl = clf.predict(self.X)


        #perform ols with our implementation
        model = RegressionMethods('ols')
        model.fit(self.X, self.z_flat)
        z_pred_model = model.predict(self.X)

        #assert difference in our implementation and sklearns
        diff = mean_squared_error(z_pred_skl, z_pred_model)
        print("Test OLS:")
        print("Max diff: ", diff)
        assert diff < self.tol


    def test_ridge(self):

        #perform ridge on the data with sklearn
        alpha = 0.1
        clf = skl.Ridge(alpha = alpha, fit_intercept=False).fit(self.X, self.z_flat)
        z_pred_skl = clf.predict(self.X)

        #perform ridge on the data with our implementation
        model = RegressionMethods(method = 'ridge', alpha = alpha)
        model.fit(self.X, self.z_flat)
        z_pred_model = model.predict(self.X)

        #assert difference
        diff = mean_squared_error(z_pred_skl, z_pred_model)
        print("Test Ridge:")
        print("Max diff: ", diff)
        assert diff < self.tol